In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel('0FLATANALYTICALexport.XLSX')

In [4]:
#correction of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'][df['Due date (-)'] == '1816-03-29 05:56:08.066277376' ] = 0
df['test'] = np.where(df['Due date (-)'] == '1970-01-01', 0, 1)

# correction of Title transfer date (-)
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype(object).where(df['Title transfer date (-)'].notnull(), 0)
df['Title transfer date (-)'] = np.where(df['Title transfer date (-)'] == 0, pd.to_datetime('today') + np.timedelta64(30,'D') , df['Title transfer date (-)'])
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype('datetime64[ns]')

#continuation of correction of Due date (-)
df['Due date (-) corrected'] = np.where(df['test'] == 1, df['Due date (-)'], df['Due date (+)'] - df['Title transfer date (+)'] + df['Title transfer date (-)'])

df['Due date (-)'] = df['Due date (-) corrected']

# converting the type of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'] = df['Due date (-)'].dt.date

df['Due date (+)'] = df['Due date (+)'].dt.date
df['Title transfer date (-)'] = df['Title transfer date (-)'].dt.date
df['Title transfer date (+)'] = df['Title transfer date (+)'].dt.date

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
# calculating the columns

df['Consumption Date'] = df['Due date (+)']
df['Consumption Type'] = 'Filled by Macros'
df['Total WC Consumption period'] = df['Due date (-)'] - df['Due date (+)']
#df['TTD influence on WC consumption'] = ''
#df['PDD influence on WC consumption'] = ''
df['TTD difference'] = df['Title transfer date (-)'] - df['Title transfer date (+)']
df['Sale Payment Terms'] = df['Due date (-)'] - df['Title transfer date (-)']
df['Buy Payment Terms'] = df['Due date (+)'] - df['Title transfer date (+)']
df['PDD difference'] = df['Sale Payment Terms'] - df['Buy Payment Terms']
df['Consumption reason'] = 'OTHER'
df['WC Amount'] = df['WC consumption amount'] / 1000000
df['Current price, USD/BBL (-)'] = ''


# converting columns to the right format
#converting into days
df['Total WC Consumption period'] = df['Total WC Consumption period'].dt.days
df['TTD difference'] = df['TTD difference'].dt.days
df['Sale Payment Terms'] = df['Sale Payment Terms'].dt.days
df['Buy Payment Terms'] = df['Buy Payment Terms'].dt.days
df['PDD difference'] = df['PDD difference'].dt.days


In [3]:
# further columns formatting, arranging
df['Company Name'] = df['Line of Business'].str[:2]
df['Company Name']

NameError: name 'df' is not defined

In [6]:
def f(row):
    if (row['Total WC Consumption period'] >= row['TTD difference'] and row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] <= row['TTD difference'] and row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['TTD difference']
    elif (row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['Total WC Consumption period']
    else:
        return 0

In [7]:
df['TTD influence on WC consumption'] = df.apply(f, axis=1)
df['PDD influence on WC consumption'] = df['Total WC Consumption period'] - df['TTD influence on WC consumption']


In [8]:
# rearrange columns
cols = df.columns.tolist()

In [9]:
cols = ['Consumption Date',
       'Consumption Type',
       'Total WC Consumption period',
        'TTD influence on WC consumption',
        'PDD influence on WC consumption',
        'TTD difference',
        'PDD difference',
        'Sale Payment Terms',
        'Buy Payment Terms',
        'Consumption reason',
        'WC Amount',
        'Buy/Sell (-)',
        'Buy/Sell (+)',
        'Due date (-)',
        'Due date (+)',
        'Title transfer date (-)',
        'Title transfer date (+)',
        'Strategy',
        'Strategy name',
        'Company Name',
        'Global Book name',
        'Line of Business',
        'Book Name',
        'Portfolio name',
        'Quantity',
        'Current amount in USD (+)',
        'Quantity BBL (ratio) (+)',
        'Current price, USD/BBL (+)',
        'Counterparty (+)',
        'Title transfer (+)',
        'Incoterms 1 (+)',
        'Incoterms 2 country (+)',
        'Incoterms 2 country (+).1',
        'Object type (+)',
        'Delivery ID (+)',
        'Internal type (+)',
        'Mirror Delivery ID (+)',
        'Current amount in USD (-)',
        'Quantity BBL (ratio) (-)',
        'Current price, USD/BBL (-)',
        'Counterparty (-)',
        'Title transfer (-)',
        'Incoterms 1 (-)',
        'Incoterms 2 country (-)',
        'Incoterms 2 country (-).1',
        'Object type (-)',
        'Delivery ID (-)',
        'Internal type (-)',
        'Mirror Delivery ID (-)',
        'Strategy start date',
        'Strategy end date',
        'Voyage ID (+)',
        'Transport name (+)',
        'Grade (+)',
        'Voyage ID (-)',
        'Transport name (-)',
        'Grade (-)']

In [11]:
# export

df_export = df[cols]

# export to excel
writer = pd.ExcelWriter('output.xlsx')
df_export.to_excel(writer,'Sheet1')
writer.save()

In [2]:
# ****************** columns to correct***************************
# WC Amount
# Company Name (first three cars from Line of Business e.g: GVA)
# Portfolio Name
# Current Amount in USD (+)
# counterparty (+)
# Current amount in USD (-)
# Current price, USD/BBL (-)
# counterparty (-)
# Incoterms 1 (-)
# Inconterms 2 country (-)
# Incoterms 2 country.1 (-)
